Imported down all the necessary libraries which is thankfully just Pandas

In [1]:
import pandas as pd

Read the dataset using Pandas read_csv feature

In [2]:
dataset=pd.read_csv('bbb_odi_data.csv')

C:\Users\Vijay\AppData\Local\Temp\ipykernel_9088\2538919985.py:1: DtypeWarning: Columns (41,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv('bbb_odi_data.csv')


Filtered out the dataset to get the instances only where the Current run rate was <=5 during the course of the game

In [3]:
resultdf=dataset[dataset['inns_rr']<=5.00]

An unnecessary column has been deleted

In [4]:
# resultdf.drop(columns='Unnamed: 0',index=1,inplace=True)

There are several columns and I chose only the ones meaningful to the related query by using a list.

In [5]:
req_cols=['bat','batruns','ballfaced']

Now, my dataset is filtered out to get only the required columns

In [6]:
result=resultdf[req_cols]

Grouped sum of the required columns gives us the overall aggregate of the data. I've reset the columns as well to include the names as a row value in the dataset.

In [7]:
grouped=result.groupby(['bat']).sum().reset_index()

In [8]:
grouped

,bat,batruns,ballfaced
0,AB de Villiers,3655,4626
1,AM Ghazanfar,5,8
2,Aamer Yamin,80,84
3,Aamir Kaleem,13,16
4,Aarif Sheikh,792,1300
...,...,...,...
1622,Zubayr Hamza,43,55
1623,Zubin Surkari,236,487
1624,Zulfiqar Babar,34,36
1625,Zulqarnain Haider,23,38


I've used this different approach to sum the dismissals because there are some run out dismissals and not always it has been the striker who has gotten out, so based on the who_out? column values summation, I've found out the number of dismissals in this specific filter. 

In [9]:
dis=pd.DataFrame(resultdf['who_out?'].value_counts()).reset_index()

This is an attempt to make a similar column in both the dataframes, and use it for joins later.

In [10]:
dis.rename({'index':'bat'},axis=1,inplace=True)

In [11]:
dis

,bat,who_out?
0,Tamim Iqbal,149
1,Hamilton Masakadza,146
2,Shakib Al Hasan,136
3,Mushfiqur Rahim,135
4,Kumar Sangakkara,126
...,...,...
1532,Ryan Flannigan,1
1533,Abhimanyu Mithun,1
1534,Sufyan Mehmood,1
1535,Prasidh Krishna,1


A left join has been performed on the left dataset which is the grouped data and then they've been joined on 'bat' column

In [12]:
finalresult=pd.merge(grouped,dis,how='left',on='bat')

Now, they've been sorted by descending order of 'batruns' and this operation has been performed in place to reflect the changes

In [13]:
finalresult.sort_values(by='batruns',ascending=False,inplace=True)

The resultant column names have been renamed for easier interpretation

In [14]:
finalresult.rename({'who_out?':'dismissals','batruns':'runs','ballfaced':'balls'},axis=1,inplace=True)

In [15]:
finalresult

,bat,runs,balls,dismissals
797,Kumar Sangakkara,5227,7873,126.0
1016,Mushfiqur Rahim,4561,6400,135.0
1562,Virat Kohli,4537,6085,94.0
856,MS Dhoni,4415,6314,95.0
1341,Shakib Al Hasan,4387,5990,136.0
...,...,...,...,...
705,Josephat Ababu,0,9,NaN
599,Ishwarjot Sohi,0,2,1.0
441,Fahad Alhashmi,0,7,3.0
1262,Ryan Rickelton,0,2,NaN


Before performing mathematical operations to find out the averages and strike rates, I've actually verified if the data types are suitable for calculations or not. Since both are of the same 'integer' type, it is easier for me to perform the operations.

In [16]:
finalresult.dtypes

bat            object
runs            int64
balls           int64
dismissals    float64
dtype: object

The averages and Runs Per Over (rpo) were calculated from the existing data and were rounded off to the 2 nearest decimal places.

In [17]:
finalresult['avg']=(finalresult['runs']/finalresult['dismissals'])
finalresult['avg']=round(finalresult['avg'],2)

In [18]:
finalresult['rpo']=(finalresult['runs']*6/finalresult['balls'])
finalresult['rpo']=round(finalresult['rpo'],2)

The end result has been saved to a csv file to use them further for visualizations on tableau

In [19]:
finalresult.to_csv('under5.csv')

In [20]:
finalresult

,bat,runs,balls,dismissals,avg,rpo
797,Kumar Sangakkara,5227,7873,126.0,41.48,3.98
1016,Mushfiqur Rahim,4561,6400,135.0,33.79,4.28
1562,Virat Kohli,4537,6085,94.0,48.27,4.47
856,MS Dhoni,4415,6314,95.0,46.47,4.20
1341,Shakib Al Hasan,4387,5990,136.0,32.26,4.39
...,...,...,...,...,...,...
705,Josephat Ababu,0,9,NaN,NaN,0.00
599,Ishwarjot Sohi,0,2,1.0,0.00,0.00
441,Fahad Alhashmi,0,7,3.0,0.00,0.00
1262,Ryan Rickelton,0,2,NaN,NaN,0.00
